In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_sample_submission.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv


# Preprocessing
## Import libraries

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [32]:
train = pd.read_csv("../input/goodreads-books-reviews-290312/goodreads_train.csv") 
test = pd.read_csv("../input/goodreads-books-reviews-290312/goodreads_test.csv")

In [33]:
print(f'Train verilerindeki satır sayısı: {train.shape[0]}')
print(f'Train verilerindeki sütun sayısı: {train.shape[1]}')
print(f'Train verilerindeki değer sayısı: {train.count().sum()}')
print(f'Train verilerinde eksik değerlere sahip satır sayısı: {sum(train.isna().sum())}')
train.head()

Train verilerindeki satır sayısı: 900000
Train verilerindeki sütun sayısı: 11
Train verilerindeki değer sayısı: 9533937
Train verilerinde eksik değerlere sahip satır sayısı: 366063


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [1]:
#train_profile = ProfileReport(train, title="Train")
#train_profile.to_notebook_iframe()

In [35]:
print(f'Test verilerindeki satır sayısı: {test.shape[0]}')
print(f'Test verilerindeki sütun sayısı: {test.shape[1]}')
print(f'Test verilerindeki değer sayısı: {test.count().sum()}')
print(f'Test verilerinde eksik değerlere sahip satır sayısı: {sum(test.isna().sum())}')
test.head()

Test verilerindeki satır sayısı: 478033
Test verilerindeki sütun sayısı: 10
Test verilerindeki değer sayısı: 4594808
Test verilerinde eksik değerlere sahip satır sayısı: 185522


,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,** spoiler alert ** \n This is definitely one ...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,"** spoiler alert ** \n ""You are what you drink...",Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,Roar is one of my favorite characters in Under...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,** spoiler alert ** \n If you feel like travel...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 stars \n I read and enjoyed the first two ...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0


## Pandas Profile

In [36]:
#test_profile = ProfileReport(test, title="test")
#test_profile.to_notebook_iframe()

# Preprocessing the review texts
# İnceleme metinlerinin ön işlenmesi

In [37]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [38]:
books = pd.concat([train.drop('rating', axis=1), test], axis=0)
books_rates = train['rating']

books.shape

(1378033, 10)

In [39]:
train.set_index("review_id", inplace=True)
test.set_index("review_id", inplace=True)

#### Gerekli olmayan bazı sütunların çıkarılması
#### Droping some non-essential columns


In [40]:
books.drop(columns=['date_updated', 'read_at', 'started_at'], axis=1, inplace=True)

#### Removing rows with negative n_votes or negative n_comments 
#### Negatif n_votes veya negatif n_comments içeren satırları kaldırma

In [41]:
books = books[(books['n_votes'] >= 0) & (books['n_comments'] >= 0)]

In [42]:
books.drop_duplicates(subset=['review_text'], inplace=True)

In [43]:
unapp_texts = books['review_text'].apply(lambda txt: np.mean(list(map(len, txt.split())))) > 25
unapp_texts.shape

(1361800,)

In [44]:
books.drop(books[unapp_texts].index, axis=0, inplace=True)
spoil_statement = 'spoiler alert'

In [45]:
books['spoil'] = [True if spoil_statement in txt else False for txt in books['review_text']]
books[books['spoil'] == True]

,user_id,book_id,review_id,review_text,date_added,n_votes,n_comments,spoil
26,8842281e1d1347389f2ab93d60773d4d,62291,e23965adda7ce7e7e415a1339e169065,** spoiler alert ** \n Loved it. The epic saga...,Tue Jul 12 12:57:07 -0700 2011,5,1,True
29,8842281e1d1347389f2ab93d60773d4d,7112495,0258b3c9a85cecb95f240e43a7642f60,** spoiler alert ** \n An interesting book in ...,Thu Jun 10 14:41:13 -0700 2010,7,3,True
32,8842281e1d1347389f2ab93d60773d4d,6411961,d44b6d28f456dbcc5b1537b4c5572400,"** spoiler alert ** \n Critics aside, Dan Brow...",Wed Sep 16 11:09:03 -0700 2009,3,3,True
34,8842281e1d1347389f2ab93d60773d4d,77432,d49eb786ef1d8557b9e6e86bcafd4570,"** spoiler alert ** \n I equally loved book 2,...",Tue May 05 18:08:22 -0700 2009,1,0,True
42,8842281e1d1347389f2ab93d60773d4d,14497,b8e08587ce1592d69548d0f644cb9893,** spoiler alert ** \n Just finished this and ...,Thu Mar 27 17:53:06 -0700 2008,6,1,True
...,...,...,...,...,...,...,...,...
477297,5ff0927d425ab1a70d2b2d41fe6b96b5,13239950,a989c818917a7463089454b356fb6671,** spoiler alert ** \n Hills like White Elepha...,Wed Oct 31 18:22:50 -0700 2012,25,7,True
477363,a16fd6b48c6e877042acbe2bda8a0007,115076,3736d5ee5904b4b5651912dd28b7ce01,** spoiler alert ** \n I was completely engros...,Mon Jun 04 07:18:34 -0700 2012,0,0,True
477777,692ce8eb4d5490980f6e74f6cb8671e2,12974372,a7777a1c18ab582586a61ad453d91949,** spoiler alert ** \n Hmm...what to say about...,Wed Nov 04 12:33:22 -0800 2015,1,0,True
477825,692ce8eb4d5490980f6e74f6cb8671e2,29044,b1e1134e7e1e90725ac4237b66657de9,** spoiler alert ** \n The first two-thirds of...,Mon Nov 24 16:03:39 -0800 2014,1,3,True


# Model Building 
# Modelleme

In [2]:
import torch
import numpy as np
import pandas as pd
from sklearn import model_selection, metrics
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer

In [3]:
class ClassificationDataset:
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        text = str(self.data["review_text"].values[item])
        target = int(self.data["rating"].values[item])
        inputs = self.tokenizer(
            text,
            max_length=512, 
            padding="max_length", 
            truncation=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "input_ids": torch.tensor(ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(target, dtype=torch.long),
        }

In [4]:
df = pd.read_csv("../input/goodreads-books-reviews-290312/goodreads_test.csv")
tokenizer = AutoTokenizer.from_pretrained("abhishek/autotrain-goodreads-review-rating-1828462866", use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained("abhishek/autotrain-goodreads-review-rating-1828462866")

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/704M [00:00<?, ?B/s]

In [5]:
df.loc[:, "rating"] = 0
dataset = ClassificationDataset(df, tokenizer)
trainer = Trainer(model)
preds = trainer.predict(dataset).predictions

***** Running Prediction *****
  Num examples = 478033
  Batch size = 8


In [ ]:
sample_submission = pd.read_csv("../input/goodreads-books-reviews-290312/goodreads_sample_submission.csv")
preds = np.argmax(preds, axis=1)

In [6]:
sample_submission.rating = preds
sample_submission.to_csv("submission2.csv", index=False)